In [1]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from all_staff import *

In [2]:
with open(obj_img_dir, 'rb') as f:
    objpoints, imgpoints = pickle.load(f)

In [3]:
class Line(object):
    def __init__(self, objpoints, imgpoints, persepctive_mode = False):
        self.do_first = True
        
        self.left_fit = None
        self.right_fit = None
        
        self.objpoints = objpoints
        self.imgpoints = imgpoints
        
        self.persepctive_mode = persepctive_mode
    
    def pipline(self, img):
        undist = undistort_img(img, self.objpoints, self.imgpoints)
        binary_img = binary_color(undist, sobel_kernel=3, gray_threshold=(45,255), color_thresold=(110,255))
        binary_img_color = np.dstack([binary_img*255, binary_img, binary_img])
        binary_warped, Minv = warp_M(binary_img)
        
        if self.do_first:
            ploty, left_fitx, right_fitx, left_fit, right_fit = slide_windows(binary_warped, nwindows=9, margin=100, minpix=50)
            self.left_fit = left_fit
            self.right_fit = right_fit
            self.do_first = False
        else:
            ploty, left_fitx, right_fitx = acc_frame_to_frame(binary_warped, self.left_fit, self.right_fit, margin = 100)
        
        if not self.persepctive_mode:
            result = img_region(binary_warped, left_fitx, right_fitx, ploty, Minv, binary_img_color)
        else:
            result = img_region_no_T(binary_warped, left_fitx, right_fitx, ploty)
            
        return result
        

In [6]:
l = Line(objpoints, imgpoints, False)

In [7]:
white_output = 'vedio/project_video_sub.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("vedio/project_video.mp4").subclip(0,5)
white_clip = clip1.fl_image(l.pipline) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video vedio/project_video_sub.mp4
[MoviePy] Writing video vedio/project_video_sub.mp4


 99%|█████████▉| 125/126 [01:33<00:00,  1.32it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: vedio/project_video_sub.mp4 

CPU times: user 1min 35s, sys: 680 ms, total: 1min 36s
Wall time: 1min 33s


In [17]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## 接下来要做

- 换threshold算法来避免光照-黑暗的场景替换
- smoothing，计算变化率，曲率，leftcenter
- 前几个frame的变化率比较